# Imports

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from random import randint
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset


In [2]:
DATA_URL = 'data/Crystal_structure.csv'

In [3]:
unprocessed_data = pd.read_csv(DATA_URL)
data = pd.read_csv(DATA_URL)
data.count()

Compound             5329
A                    5329
B                    5329
In literature        5329
v(A)                 5329
v(B)                 5329
r(AXII)(Å)           5329
r(AVI)(Å)            5329
r(BVI)(Å)            5329
EN(A)                5329
EN(B)                5329
l(A-O)(Å)            5329
l(B-O)(Å)            5329
ΔENR                 5329
tG                   5329
τ                    5329
μ                    5329
Lowest distortion    5329
dtype: int64

In [4]:
data.head()

,Compound,A,B,In literature,v(A),v(B),r(AXII)(Å),r(AVI)(Å),r(BVI)(Å),EN(A),EN(B),l(A-O)(Å),l(B-O)(Å),ΔENR,tG,τ,μ,Lowest distortion
0,Ac2O3,Ac,Ac,False,0,0,1.12,1.12,1.12,1.1,1.10,0.0,0.000000,-3.248000,0.707107,-,0.800000,cubic
1,AcAgO3,Ac,Ag,False,0,0,1.12,1.12,0.95,1.1,1.93,0.0,2.488353,-2.565071,0.758259,-,0.678571,orthorhombic
2,AcAlO3,Ac,Al,False,0,0,1.12,1.12,0.54,1.1,1.61,0.0,1.892894,-1.846714,0.918510,-,0.385714,cubic
3,AcAsO3,Ac,As,False,0,0,1.12,1.12,0.52,1.1,2.18,0.0,1.932227,-1.577429,0.928078,-,0.371429,orthorhombic
4,AcAuO3,Ac,Au,False,0,0,1.12,1.12,0.93,1.1,2.54,0.0,2.313698,-2.279786,0.764768,-,0.664286,orthorhombic


# Data cleaning and pre-processing

## Unnecessary features

The "present in literature" field should not be determinant of the crystal structure.

In [5]:
data = data.drop(["In literature"], axis=1)
data.columns

Index(['Compound', 'A', 'B', 'v(A)', 'v(B)', 'r(AXII)(Å)', 'r(AVI)(Å)',
       'r(BVI)(Å)', 'EN(A)', 'EN(B)', 'l(A-O)(Å)', 'l(B-O)(Å)', 'ΔENR', 'tG',
       'τ', 'μ', 'Lowest distortion'],
      dtype='object')

Likewise, the name of the compound should not be fed to the model. Plus, the information in the name is already contained in the `A` and `B` fields. The compound name is always `AB` followed by `O3`

In [6]:
data = data.drop(["Compound"], axis=1)
data.columns

Index(['A', 'B', 'v(A)', 'v(B)', 'r(AXII)(Å)', 'r(AVI)(Å)', 'r(BVI)(Å)',
       'EN(A)', 'EN(B)', 'l(A-O)(Å)', 'l(B-O)(Å)', 'ΔENR', 'tG', 'τ', 'μ',
       'Lowest distortion'],
      dtype='object')

## Dash pre-processing

In [7]:
(data == "-").any()

A                    False
B                    False
v(A)                  True
v(B)                  True
r(AXII)(Å)           False
r(AVI)(Å)            False
r(BVI)(Å)            False
EN(A)                False
EN(B)                False
l(A-O)(Å)            False
l(B-O)(Å)            False
ΔENR                 False
tG                   False
τ                     True
μ                    False
Lowest distortion     True
dtype: bool

### Valencies `v(A)` and `v(B)`

In [8]:
print(data["v(A)"].unique())
print(data["v(B)"].unique())

['0' '-' '1' '3' '2' '4' '5']
['0' '-' '5' '3' '4' '2' '1']


In [9]:
(data["v(A)"] == "-").sum(), (data["v(B)"] == "-").sum()

(1881, 1881)

In [10]:
((data["v(A)"] == "-").sum() and (data["v(B)"] == "-").sum()).sum()

1881

In [11]:
data = data.drop(data[data["v(A)"] == "-"].index)
data = data.drop(data[data["v(B)"] == "-"].index)

In [12]:
valency_a_one_hot = pd.get_dummies(data['v(A)'], prefix="v(A)=", prefix_sep="")
valency_b_one_hot = pd.get_dummies(data['v(B)'], prefix="v(B)=", prefix_sep="")
data = pd.concat([data, valency_a_one_hot, valency_b_one_hot], axis=1)

Now the dataframe has the valencies one-hot encoded and we can safely drop the original columns

In [13]:
data = data.drop(["v(A)"], axis=1)
data = data.drop(["v(B)"], axis=1)
data.columns

Index(['A', 'B', 'r(AXII)(Å)', 'r(AVI)(Å)', 'r(BVI)(Å)', 'EN(A)', 'EN(B)',
       'l(A-O)(Å)', 'l(B-O)(Å)', 'ΔENR', 'tG', 'τ', 'μ', 'Lowest distortion',
       'v(A)=0', 'v(A)=1', 'v(A)=2', 'v(A)=3', 'v(A)=4', 'v(A)=5', 'v(B)=0',
       'v(B)=1', 'v(B)=2', 'v(B)=3', 'v(B)=4', 'v(B)=5'],
      dtype='object')

In [14]:
len(data["τ"].unique())

1608

In [15]:
data = data.drop(["τ"], axis=1)
data.columns

Index(['A', 'B', 'r(AXII)(Å)', 'r(AVI)(Å)', 'r(BVI)(Å)', 'EN(A)', 'EN(B)',
       'l(A-O)(Å)', 'l(B-O)(Å)', 'ΔENR', 'tG', 'μ', 'Lowest distortion',
       'v(A)=0', 'v(A)=1', 'v(A)=2', 'v(A)=3', 'v(A)=4', 'v(A)=5', 'v(B)=0',
       'v(B)=1', 'v(B)=2', 'v(B)=3', 'v(B)=4', 'v(B)=5'],
      dtype='object')

### Target `Lowest distortion` 

This is the target field so any pre-processing could be bad for accuracy of the model we want to develop. Since the dash `-` it's only present in 53 rows we could safely remove these rows from the dataset, as it's approximately a 1% of the data. 


In [16]:
(data["Lowest distortion"] == "-").sum()

22

## Change categorical data into numerical
Here comes again the one-hot encoding. The fields where this is necessary are `A` and `B`, which show the first and second elements in the compound. 
The target `Lowest distortion` is also categorical but must be encoded to integer values

### Features

In [17]:
a_one_hot = pd.get_dummies(data['A'], prefix="A=", prefix_sep="")
b_one_hot = pd.get_dummies(data['B'], prefix="B=", prefix_sep="")
data = pd.concat([data, a_one_hot, b_one_hot], axis=1)

In [18]:
data = data.drop(["A"], axis=1)
data = data.drop(["B"], axis=1)

### Target `Lowest distortion`

In [19]:
unknown_data = data[data["Lowest distortion"] == "-"]
data = data.drop(data[data["Lowest distortion"] == "-"].index)

In [20]:
class_names = data["Lowest distortion"].unique()
class_names

array(['cubic', 'orthorhombic', 'rhombohedral', 'tetragonal'],
      dtype=object)

In [21]:
data["Lowest distortion"] = data["Lowest distortion"].astype('category')
data["Lowest distortion"] = data["Lowest distortion"].cat.codes

## Split the train/test/unknown data

In [22]:
len(data), len(unknown_data)

(3426, 22)

In [23]:
features = data.drop(labels=["Lowest distortion"], axis=1)
target = data["Lowest distortion"]

In [24]:
SEED = 0
X_temp, X_test, y_temp, y_test = train_test_split(features, target, test_size=0.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=SEED)


## Preprocessed dataset

In [25]:
data

,r(AXII)(Å),r(AVI)(Å),r(BVI)(Å),EN(A),EN(B),l(A-O)(Å),l(B-O)(Å),ΔENR,tG,μ,...,B=Ti,B=Tl,B=Tm,B=U,B=V,B=W,B=Y,B=Yb,B=Zn,B=Zr
0,1.12,1.12,1.12,1.10,1.10,0.00000,0.000000,-3.248000,0.707107,0.800000,...,False,False,False,False,False,False,False,False,False,False
1,1.12,1.12,0.95,1.10,1.93,0.00000,2.488353,-2.565071,0.758259,0.678571,...,False,False,False,False,False,False,False,False,False,False
2,1.12,1.12,0.54,1.10,1.61,0.00000,1.892894,-1.846714,0.918510,0.385714,...,False,False,False,False,False,False,False,False,False,False
3,1.12,1.12,0.52,1.10,2.18,0.00000,1.932227,-1.577429,0.928078,0.371429,...,False,False,False,False,False,False,False,False,False,False
4,1.12,1.12,0.93,1.10,2.54,0.00000,2.313698,-2.279786,0.764768,0.664286,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,0.89,0.72,0.60,1.33,1.90,2.38342,0.000000,-1.678786,0.809637,0.428571,...,False,False,False,False,False,False,False,False,False,False
5319,0.89,0.72,0.61,1.33,1.54,2.38342,1.927849,-1.813036,0.805609,0.435714,...,True,False,False,False,False,False,False,False,False,False
5322,0.89,0.72,0.76,1.33,1.38,2.38342,2.047800,-2.161214,0.749664,0.542857,...,False,False,False,True,False,False,False,False,False,False
5323,0.89,0.72,0.54,1.33,1.63,2.38342,1.758039,-1.645679,0.834678,0.385714,...,False,False,False,False,True,False,False,False,False,False


# Random Forest algorithm

In [26]:
forest = RandomForestClassifier(n_estimators=1000, random_state=SEED, verbose=1)
forest = forest.fit(X_train, y_train)
forest.verbose = 0 # silence further control msg

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    3.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    6.6s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:   10.5s


In [27]:
accuracy = round(forest.score(X_test, y_test) * 100, 2)
print(f"Accuracy of the random forest: {accuracy}%")

Accuracy of the random forest: 81.92%


# Deep neural net approach

In [28]:

# Assuming X_train, y_train, X_test, y_test are already defined
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values.tolist(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values.tolist(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
X_val_tensor = torch.tensor(X_val.values.tolist(),dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.long)


# Create DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the PyTorch model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size1=300, hidden_size2=200, hidden_size3=100, output_size=4):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

# Instantiate the model
INPUT_SHAPE = X_train.shape[1]
model = Net(INPUT_SHAPE)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training loop with validation
num_epochs = 100
best_val_acc = 0.0
save_path = 'best_model.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)
    
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_corrects += torch.sum(preds == labels.data)
    val_loss /= len(val_loader.dataset)
    val_acc = val_corrects.double() / len(val_loader.dataset)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')

# Load the best model for evaluation
model.load_state_dict(torch.load(save_path))
model.eval()

# Evaluate the model
test_loss = 0.0
test_corrects = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_corrects += torch.sum(preds == labels.data)
test_loss /= len(test_loader.dataset)
test_acc = test_corrects.double() / len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')


Epoch 1/100, Loss: 1.0436, Accuracy: 0.6190, Val Loss: 0.9077, Val Accuracy: 0.6234
Epoch 2/100, Loss: 0.9098, Accuracy: 0.6292, Val Loss: 0.8740, Val Accuracy: 0.6234
Epoch 3/100, Loss: 0.8600, Accuracy: 0.6311, Val Loss: 0.8190, Val Accuracy: 0.6234
Epoch 4/100, Loss: 0.8022, Accuracy: 0.6618, Val Loss: 0.8062, Val Accuracy: 0.6584
Epoch 5/100, Loss: 0.7432, Accuracy: 0.6983, Val Loss: 0.7436, Val Accuracy: 0.6978
Epoch 6/100, Loss: 0.6965, Accuracy: 0.7241, Val Loss: 0.7079, Val Accuracy: 0.7197
Epoch 7/100, Loss: 0.6552, Accuracy: 0.7382, Val Loss: 0.6878, Val Accuracy: 0.7343
Epoch 8/100, Loss: 0.6102, Accuracy: 0.7616, Val Loss: 0.6703, Val Accuracy: 0.7431
Epoch 9/100, Loss: 0.6057, Accuracy: 0.7669, Val Loss: 0.6639, Val Accuracy: 0.7562
Epoch 10/100, Loss: 0.5653, Accuracy: 0.7796, Val Loss: 0.6526, Val Accuracy: 0.7445
Epoch 11/100, Loss: 0.5424, Accuracy: 0.7898, Val Loss: 0.6502, Val Accuracy: 0.7431
Epoch 12/100, Loss: 0.5024, Accuracy: 0.8092, Val Loss: 0.6278, Val Accura

In [41]:
print(f'Accuracy of Deep Learning Model is {round(test_acc.detach().numpy()*100,2)}%')

Accuracy of Deep Learning Model is 81.34%


# Conclusions

Both random forest and a deep neural net result in very similar accuracy on unseen images. The DNN can be trained for extra epoches but after the 25th epoch the risk of overfitting is starting to show up.

Perhaps this is the plateau with the amount and quality of data we have, but I'll be have to be proven otherwise. For the moment let's wrap this up by trying the model on the items we extracted from the dataset due to the target value missing.

In [45]:
features = unknown_data.drop(labels=["Lowest distortion"], axis=1)
features_tensor = torch.tensor(features.values.tolist(), dtype=torch.float32)

predictions = model(features_tensor).detach().numpy().argmax(axis=1)
prediction_df = pd.DataFrame(columns=["Compound", "Predicted Structure"])
i = 0
for row in features.iterrows():
    row_index = row[0]
    item_dict = {'Compound': [unprocessed_data.iloc[row_index]["Compound"]], 'Predicted Structure': [class_names[predictions[i]]]}
    item = pd.DataFrame(item_dict)
    prediction_df = pd.concat([prediction_df, item], ignore_index=True)
    i += 1

In [46]:
prediction_df

,Compound,Predicted Structure
0,FeLuO3,cubic
1,GdCoO3,orthorhombic
2,LaNpO3,orthorhombic
3,NbNpO3,cubic
4,NiNpO3,cubic
5,NiOsO3,cubic
6,NiTcO3,cubic
7,NpCsO3,orthorhombic
8,Np2O3,cubic
9,OsCaO3,cubic
